In [2]:
%pylab inline

from xgboost.sklearn import XGBClassifier
from typing import Tuple
import xgboost as xgb
import pandas as pd
import json
import sklearn

dataset = pd.read_csv('data/multisession-eeg.csv')
fromstring = lambda array_str: np.fromstring(array_str, dtype=float, sep=',')
dataset.raw_fft = dataset.raw_fft.apply(fromstring)
# dataset.raw_fft.iloc[0]

Populating the interactive namespace from numpy and matplotlib


# Passthoughts

What if you could simply *think your password*? That's the premise behind *passthoughts*. We'll discuss passthoughts in more depth in lecture 3, but for now, we'll lay this out as a classification problem:

> Given a reading, and a person, is that person who they claim to be?

We'll structure this problem as follows: For each subject, we'll train a classifier. That subject's readings will be positive example, and everyone else's readings will be negative examples.

We can make this a little fancier by having people use specific thoughts (e.g. "focus on your breathing," "sing a song in your head," etc). We'll make sure our methods can handle this case, but for the time being, we'll just use the `"unabeled"` readings - people doing nothing in particular.

We'll use subject `A` as our "target" individual. We will train on this subject for this assignment, and train against the other subjects in the corpus (subjects `B` and `C`).

In [3]:
def to_matrix (series):
    return np.array([ x for x in series ])

def readings_right_subject_right_task (subj, task, session=0):
    return to_matrix(dataset[
        (dataset['subject'] == subj) &
        (dataset['label'] == task) &
        (dataset['session'] == session)
    ].raw_fft)

def readings_wrong_subj_any_task (subj):
    return to_matrix(dataset[
        (dataset['subject'] != subj)
    ].raw_fft)

In [22]:
positive = readings_right_subject_right_task('A', 'unlabeled', 0)
negative = readings_wrong_subj_any_task('A')
positive.shape, negative.shape

((40, 516), (1228, 516))

## TODO

Notice how we structured our positive and negative examples:

- *Positive examples*: The right person thinking the right task.

- *Negative examples*: The wrong person thinking any task (whether it is right or wrong).

In the context of passthoughts, consider other possibilites for selecting positive and negative features. Here, (1) pick one configuration of positive and negative examples, aside from the ones listed, and (2) discuss their possible consequences (pros/cons). Explain how you might evaluate this selection (with data, with user experiments, etc - your choice).

In [23]:
dataset['label'].unique()

array(['unlabeled', 'breathe', 'song', 'song_o', 'sport', 'breathe_o',
       'speech', 'face', 'calibration', 'word_x', 'phrase_x', 'face_x',
       'breatheopen', 'song_x', 'sport_x', 'breatheclosed', 'word_c',
       'phrase_c', 'face_c', 'song_c', 'sport_c'], dtype=object)

In [42]:
positive = readings_right_subject_right_task('A', 'face', 0)
negative = readings_wrong_subj_any_task('A')
positive.shape, negative.shape

((23, 516), (1228, 516))

"Face" seems like an interesting "mental task" because the face is something specific. For example, with a sport like "swimming," my mental picture of swimming might be pretty different than someone else's. Maybe when I think of swimming I think of doing laps in a pool but when someone else thinks of swimming, they think of a shark in the ocean. These mental pictures are pretty different. In contrast, if someone shows me a picture of a face and shows that picture to someone else and asks us both to close our eyes and picture the face, we should theoretically be imagining pretty close to the same thing.

This similarity in mental image could increase the false acceptance rate for this passthought. We can try to test this by comparing the FAR of this passthought to other, potentially more subjective, passthoughts in the dataset.

Now, we'll turn these data into our feature/label matrices `X` and `y`.

In [43]:
X = np.concatenate([positive, negative])

In [44]:
y = np.array([ 0 for x in positive] + [ 1 for x in negative])
assert X.shape[0] == y.shape[0]

Note that we are assigning `0` to "positive" examples, and `1` to "negative" examples. That means `0` will mean "ACCEPT" and `1` will mean "REJECT."

## TODO

Now, train and test a classifier! Estimate your classifier's accuracy.

In [45]:
# Your code here....

def fresh_clf () -> XGBClassifier:
    return XGBClassifier(
        # Don't worry about those parameters for now,
        # though feel free to look them up if you're interested.
        objective= 'binary:logistic',
        seed=27)

def xgb_cross_validate (
    X: np.array,
    y: np.array,
    nfold: int=7
) -> Tuple[XGBClassifier, pd.DataFrame]:
    # eval_metrics:
    # http://xgboost.readthedocs.io/en/latest//parameter.html
    metrics = ['error@0.1', 'auc']
#     metrics = [ 'auc' ]
    # we use the @ syntax to override the default of 0.5 as the threshold for 0 / 1 classification
    # the intent here to to minimize FAR at the expense of FRR
    alg = fresh_clf()
    xgtrain = xgb.DMatrix(X,y)
    param = alg.get_xgb_params()
    cvresults = xgb.cv(param,
                      xgtrain,
                      num_boost_round=alg.get_params()['n_estimators'],
                      nfold=nfold,
                      metrics=metrics,
                      early_stopping_rounds=100
                      )
    alg.set_params(n_estimators=cvresults.shape[0])
    alg.fit(X,y,eval_metric=metrics)
    return alg, cvresults

In [46]:
X_train, X_validate, y_train, y_validate = sklearn.model_selection.train_test_split(
    X, y, 
    test_size=0.33, 
    random_state=42)

clf, cvres = xgb_cross_validate(X_train, y_train)

In [47]:
clf.score(X_validate, y_validate)

0.99515738498789341

For authentication, what we want even more than "accuracy" here are two metrics:

- False Acceptance Rate (FAR): The percentage of readings *not* from subject A incorrectly classified "ACCEPT."
- False Rejection Rate (FRR): The percentage of readings *from* subject A incorrectly classified 'REJECT."

For authentication /security/, we want FAR to be as low as possible (so nobody can break in).
For authentication /usability/, we want FRR to be low (so user's don't get frustrated constantly re-trying their passthought).

In [76]:
def far_frr (classifier, features, labels):
    # predict all the labels
    y_pred = classifier.predict(features)
    false_accepts = 0
    false_rejects = 0
    for predicted, actual in zip(y_pred, labels):
        # if we should have rejected,
        # but in fact accepted,
        if (actual == 1) and (predicted == 0):
            # increment false accepts
            false_accepts += 1
        # if we should have accepted,
        # but in fact rejected,
        if (actual == 0) and (predicted == 1):
            # increment false rejections
            false_rejects += 1
    # calculate proportions for each
    filterFor = lambda x, lst: len(list(filter(lambda y: y==x, lst)))
    far = false_accepts / filterFor(0, y_pred)
    frr = false_rejects / filterFor(1, y_pred)
    return far, frr

In [78]:
far, frr = far_frr(clf, X_validate, y_validate)
f'FAR: {far} - FRR: {frr}'

'FAR: 0.16666666666666666 - FRR: 0.002457002457002457'

Now, these results might be good. 

But our classifier's accuracy could be misleading.   

Can you see why? 

# Nonstationarity

We are training, and testing, using data recorded over a single session. As we know, EEG changes over time, a property known as *nonstationarity*. Will our great results still hold a few weeks later?

Let's take subject `A`'s data from sessions 1 and 2, which were recorded a few weeks after session 0.

session0: 'unlabeled', 'breathe', 'song', 'song_o', 'sport', 'breathe_o', 'speech', 'face'

session1: 'unlabeled', 'calibration', 'word_x', 'phrase_x', 'face_x', 'breatheopen', 'song_x', 'sport_x'

session2: 'unlabeled', 'calibration', 'breatheclosed', 'word_x', 'word_c', 'phrase_x', 'phrase_c', 'face_x', 'face_c', 'breatheopen', 'song_x', 'song_c', 'sport_x', 'sport_c'

In [79]:
X_subja_sess1 = readings_right_subject_right_task('A', 'face_x', 1)
X_subja_sess2 = readings_right_subject_right_task('A', 'face_x', 2)
X_subja_later = np.concatenate([X_subja_sess1, X_subja_sess2])
y_subja_later = [ 0 for x in X_subja_later ]
session_2 = dataset[
        (dataset['subject'] == 'A')
    ].groupby(['label', 'session'])
session_2.head(1)


,time,subject,session,label,raw_fft
0,2017-07-22T20:37:13.267775811Z,A,0,unlabeled,"[10.3113040924, 14.77069664, 12.213514328, 9.7..."
7,2017-07-22T20:37:20.310935713Z,A,0,breathe,"[14.7440872192, 20.4090538025, 17.9200210571, ..."
41,2017-07-22T20:37:37.807324489Z,A,0,song,"[-2.76828026772, 3.32925319672, 8.70094108582,..."
68,2017-07-22T20:37:54.684981624Z,A,0,song_o,"[-5.90404844284, 0.217925652862, -0.3990439474..."
97,2017-07-22T20:38:12.736269786Z,A,0,sport,"[7.99157047272, 11.9106330872, 8.64755058289, ..."
126,2017-07-22T20:38:33.093049999Z,A,0,breathe_o,"[3.60893774033, 7.91881132126, 4.46731090546, ..."
161,2017-07-22T20:38:59.995398858Z,A,0,speech,"[5.0061416626, 12.413517952, 15.9103908539, 12..."
189,2017-07-22T20:39:10.794270967Z,A,0,face,"[11.7602720261, 17.6141681671, 14.2930927277, ..."
222,2017-07-22T20:41:48.998198485Z,A,1,unlabeled,"[5.43575334549, 11.4954481125, 12.8636674881, ..."
231,2017-07-22T20:41:57.451022051Z,A,1,calibration,"[18.5128517151, 23.5231590271, 24.8949813843, ..."


Now, let's try the classifier we trained on the original data, testing it on the later data.

In [80]:
far, frr = far_frr(clf, X_subja_later, y_subja_later)
f'FAR: {far*100}% - FRR: {frr*100}%'

'FAR: 0.0% - FRR: 100.0%'

As we will discuss more in lecture 3, this is a problem for us. After all, we can calibrate our target subject, but we then expect them to leave the lab and go use the device later on. If their state changes so much that they can no longer be authenticated, we can't very well claim our system is accurate!

## TODO

The crux of the lab focuses on nonstationarity. At minimum, your mission is to quantify and qualify *what* is changing in EEG signals over time. You may use any tools in answering this question.

You also have your choice of corpus:

- Study subject `A`'s recordings over the three sessions provided here.
- Study one subject's recordings over the course of a year.

You can use both of these corpora, if you would like.

Some questions to spur investigation:

- What features of readings cause a classifier that works on earlier recordings fail on later ones?
- What features remain the same? Are there any?
- What might be the source of these changing features? Changing placement in the EEG device? Changing properties of the brain?

Please note below all work you do, and any notes you make along the way. Ideally, your work should read like a story - words (and questions!) interspersed with code. Good luck, and have fun!